## Importing all the dependencies

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import nltk
from bs4 import BeautifulSoup
import urllib
from nltk.tokenize import sent_tokenize
from nltk.corpus import cmudict
from nltk.corpus import stopwords

## Importing the ciklist into Pandas dataframe and doing some preliminary analysis

In [7]:
df = pd.read_excel(r'D:\nlp_internship\Data Science\cik_list.xlsx')

In [8]:
df.shape

(152, 6)

In [9]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [10]:
df.describe()

,CIK,FYRMO
count,152.000000,152.000000
mean,5861.605263,200220.750000
std,2310.633436,429.991117
min,3662.000000,199402.000000
25%,4447.000000,199906.000000
50%,5907.000000,200101.500000
75%,6201.000000,200605.500000
max,12239.000000,201407.000000


## we need to extract the files from the web, hence hence lets add the hyperlink and extract all the text data.

In [11]:

hyperlink = 'https://www.sec.gov/Archives/'
df['SECFNAME'] = hyperlink + df['SECFNAME']

In [12]:
df['SECFNAME'][2] # 

'https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt'

In [8]:
# function to find number of syllables in the word
from nltk.corpus import cmudict
phoneme_dictonary = dict(cmudict.entries())
def syllables_count(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for i in range(1,len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            count += 1
    if word[-1] == 'e':
        count -= 1
    if word[-2:] == 'le':
        count += 1
    if count == 0:
        count += 1
    # comparing the values with cmudict
    try:
        second_count = 0
        for val in phoneme_dictonary[word.lower()]:
            for val1 in val:
                if val1[-1].isdigit():
                    second_count += len(val1)
        return second_count
    except KeyError: # this is in case the values are not found in the cmu dict
        return count

    

    


In [9]:
mgt_disc_ana = r"item[^a-zA-Z\n]*\d\s*\.\s*management\'s discussion and analysis.*?^\s*item[^a-zA-Z\n]*\d\s*\.*"
qlty_qnt_disc = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Quantitative and Qualitative Disclosures about " \
            r"Market Risk.*?^\s*item\s*\d\s*"
rsk_fct = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Risk Factors.*?^\s*item\s*\d\s*"

In [10]:
constrain_dict = pd.read_excel(r'D:\nlp_internship\Data Science\constraining_dictionary.xlsx')
uncertainty_dict = pd.read_excel(r'D:\nlp_internship\Data Science\uncertainty_dictionary.xlsx')

In [ ]:
# creating a dataframe of positive and negative words
negative_words = master_dict[master_dict['Negative'] == 2009]['Word']
negative_words = negative_words.str.lower() # converting into lowercase
# creating a dataframe of positive and negative words
positive_words = master_dict[master_dict['Positive'] == 2009]['Word']
positive_words = positive_words.str.lower() # converting into lowercase

In [ ]:
# comparing with stopwords to remove the words
negative_dict = []
for val in stop_words[0].to_list():
    for val1 in negative_words.to_list():
        if val != val1:
            negative_dict.append(val1)
# comparing with stopwords to remove the words
positive_dict = []
for val in stop_words[0].to_list():
    for val1 in positive_words.to_list():
        if val != val1:
            positive_dict.append(val1)

In [22]:
def preliminary_cleaner(text):
    if text:
        array = []
        soup =text
        soup = re.sub('[\d%/$]','',str(soup))
        soup = re.sub("\\\\n",'',soup)
        soup = ' '.join(soup.split())
        return soup
        
    else:
        return None

In [23]:

array_one,array_two,array_three = [],[],[]
for hyperlink in df['SECFNAME']:
    text = urllib.request.urlopen(hyperlink).read().decode('utf-8')
    matches_mda = re.findall(mgt_disc_ana, text, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    matches_qlty_qnt_disc= re.findall(qlty_qnt_disc,text,re.IGNORECASE | re.DOTALL | re.MULTILINE)
    matches_rsk_fct = re.findall(rsk_fct,text,re.IGNORECASE | re.DOTALL | re.MULTILINE) 
    array_one.append(preliminary_cleaner(matches_mda))
    array_two.append(preliminary_cleaner(matches_qlty_qnt_disc))
    array_three.append(preliminary_cleaner(matches_rsk_fct))
''' if matches_mda:
    soup = matches_mda
    soup = re.sub('[\d%/$]','',str(soup))
    soup = re.sub("\\\\n",'',soup)
    soup = ' '.join(set(soup.split()))
    array_one.append(soup)

else:
    array_one.append(None)'''
    
''' if matches_qlty_qnt_disc:
    soup = matches_qlty_qnt_disc
    soup = re.sub('[\d%/$]','',str(soup))
    soup = re.sub("\\\\n",'',soup)
    soup = ' '.join(set(soup.split()))
    array_two.append(soup)

else:
    array_two.append(None)'''

''' if matches_rsk_fct:
    soup = matches_rsk_fct
    soup = re.sub('[\d%/$]','',str(soup))
    soup = re.sub("\\\\n",'',soup)
    soup = ' '.join(set(soup.split()))
    array_three.append(soup)
else:
    array_three.append(None)'''
    
    
    
    
   



      


' if matches_rsk_fct:\n    soup = matches_rsk_fct\n    soup = re.sub(\'[\\d%/$]\',\'\',str(soup))\n    soup = re.sub("\\\\n",\'\',soup)\n    soup = \' \'.join(set(soup.split()))\n    array_three.append(soup)\nelse:\n    array_three.append(None)'

In [24]:
df1,df2,df3 = pd.DataFrame(array_one),pd.DataFrame(array_two),pd.DataFrame(array_three)
merged_dataframe = pd.concat([df1,df2,df3],axis = 1)
merged_dataframe.columns = ['matches_mda','matches_qlty_qnt_disc','matches_rsk_fct']
merged_dataframe.to_csv('merged_dataframe_all.csv')

In [25]:
df

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,https://www.sec.gov/Archives/edgar/data/12239/...
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/12239/...
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,https://www.sec.gov/Archives/edgar/data/12239/...
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,https://www.sec.gov/Archives/edgar/data/12239/...


In [14]:
df2 = pd.read_csv(r'D:\nlp_internship\submission\merged_dataframe_all.csv')

In [15]:
final_dataframe = pd.concat([df,df2],axis = 1)

In [16]:
final_dataframe.drop(columns='Unnamed: 0',inplace = True)

In [17]:
# 1 Sentiment Analysis
    # 1.1 Removing all stop words
stop_words = pd.read_csv(r'D:\nlp_internship\Data Science\stop_words\StopWords_GenericLong.txt',header=None)
stopword_list = stop_words[0].to_list()
master_dict = pd.read_csv(r'D:\nlp_internship\Data Science\stop_words\LoughranMcDonald_MasterDictionary_2018.csv')
# creating dictonary of positive and negative words
# creating a dataframe of positive and negative words
negative_words = master_dict[master_dict['Negative'] == 2009]['Word']
negative_words = negative_words.str.lower() # converting into lowercase
# creating a dataframe of positive and negative words
positive_words = master_dict[master_dict['Positive'] == 2009]['Word']
positive_words = positive_words.str.lower() # converting into lowercase


In [19]:
from nltk.tokenize import wordpunct_tokenize
final_dataframe.fillna(' ',inplace = True)


In [20]:
#letters_only = re.sub("[^a-zA-Z]"," ",lower_case) 
final_dataframe['matches_mda'] = final_dataframe['matches_mda'].apply(lambda x : re.sub("[^a-zA-Z]"," ",x)) # removing everything except letters
final_dataframe['matches_mda'] = final_dataframe['matches_mda'].apply(lambda x : wordpunct_tokenize(x)) # tokenizing the frame

In [21]:
# cleaning stopwords
final_dataframe['matches_mda'] = final_dataframe['matches_mda'].apply(lambda x : [word.lower() for word in x if word not in stopword_list])

In [39]:
final_dataframe['matches_mda'].apply(lambda x : len([word for word in x if word in negative_dict]))

0    62
1    46
2     0
Name: matches_mda, dtype: int64

In [37]:
len([word for word in final_dataframe.matches_mda[0] if word in positive_dict])

17

In [190]:
def positive_score(variable,data_frame):
    return data_frame[variable].apply(lambda x : len([word for word in x if word in positive_dict]))
def negative_score(variable,data_frame):
    return data_frame[variable].apply(lambda x : len([word for word in x if word in negative_dict]))
def polarity_score(positive_words_dataframe,negative_words_dataframe):
    val1 = (positive_words_dataframe - negative_words_dataframe)
    val2 = (positive_words_dataframe + negative_words_dataframe + 0.00001)
    return val1 / val2
def average_sentence_length(column_with_words,column_with_sentences):
    words = column_with_words.apply(lambda x : len([word for word in x]))
    sentences = column_with_sentences.apply(lambda x : len(sent_tokenize(str(x))))
    return words / sentences
phoneme_dictonary = dict(cmudict.entries())
def syllables_count(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for i in range(1,len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            count += 1
    if word[-1] == 'e':
        count -= 1
    if word[-2:] == 'le':
        count += 1
    if count == 0:
        count += 1
    # comparing the values with cmudict
    try:
        second_count = 0
        for val in phoneme_dictonary[word.lower()]:
            for val1 in val:
                if val1[-1].isdigit():
                    second_count += len(val1)
        return second_count
    except KeyError: # this is in case the values are not found in the cmu dict
        return count
def complex_words(column_with_words):
    complex_words = column_with_words.apply(lambda x : len([word for word in x if syllables_count(word) > 2]))
    words = column_with_words.apply(lambda x : len([word for word in x]))
    ans = complex_words / words
    ans = ans.fillna(0)
    return ans

def fog_index(column_with_words,column_with_sentences):
    return 0.4 * (average_sentence_length(column_with_words,column_with_sentences) + complex_words(column_with_words))

def complex_words_count(column_with_words):
    complex_words = column_with_words.apply(lambda x : len([word for word in x if syllables_count(word) > 2]))
    ans = complex_words 
    ans = ans.fillna(0)
    return ans
def word_counts(column_with_words):
    stopwords_nltk = np.array(list(set(stopwords.words('english'))))
    words = column_with_words.apply(lambda x : len([word for word in x if word not in stopwords_nltk]))
    ans =  words
    ans = ans.fillna(0)
    return ans

def uncertainty_words_count(column_with_words):
    uncertainty_dict = pd.read_excel(r'D:\nlp_internship\Data Science\uncertainty_dictionary.xlsx')
    uncertainty_dict = uncertainty_dict['Word'].str.lower().to_list()
    words = column_with_words.apply(lambda x : len([word for word in x if word  in uncertainty_dict]))
    return words
def constraning_words_count(column_with_words):
    constrain_dict = pd.read_excel(r'D:\nlp_internship\Data Science\constraining_dictionary.xlsx')
    constrain_dict = constrain_dict['Word'].str.lower().to_list()
    words = column_with_words.apply(lambda x : len([word for word in x if word  in constrain_dict]))
    return words
def positive_words_proportion(positive_words_column,column_with_words):
    words = column_with_words.apply(lambda x : len([word for word in x]))
    ans = positive_words_column / words
    ans = ans.fillna(0)
    return ans
def negative_words_proportion(negative_words_column,column_with_words):
    words = column_with_words.apply(lambda x : len([word for word in x]))
    ans = negative_words_column / words
    ans = ans.fillna(0)
    return ans
def uncertaninty_words_proportion(column_with_words):
    words = column_with_words.apply(lambda x : len([word for word in x]))
    ans = uncertainty_words_count(column_with_words) / words
    ans = ans.fillna(0)
    return ans
def constraining_words_proportion(column_with_words):
    words = column_with_words.apply(lambda x : len([word for word in x]))
    ans = constraning_words_count(column_with_words) / words
    ans = ans.fillna(0)
    return ans




